# LOADING DATASET

In [66]:
!pip install zstandard
!pip install chess

In [33]:
import pathlib
import urllib
import zstandard
import chess
import torch
import numpy as np
from torch import nn
import math
import time

In [2]:
def __download(url: str, name: str) -> str:
    path, _ = urllib.request.urlretrieve(url, name)
    return path


def __unpack(path: str, name: str):
    input_file = pathlib.Path(path)
    with open(input_file, 'rb') as compressed:
        decomp = zstandard.ZstdDecompressor()
        output_path = name
        with open(output_path, 'wb') as destination:
            decomp.copy_stream(compressed, destination)
            destination.close()
        compressed.close()


def __remove(path: str):
    pathlib.Path.unlink(pathlib.Path(path))

In [3]:
path = __download("https://database.lichess.org/lichess_db_puzzle.csv.zst", "lichess_db_puzzle.csv.zst")

In [4]:
__unpack(path, "lichess_db_puzzle.csv")

In [5]:
__remove("lichess_db_puzzle.csv.zst")

In [6]:
class Puzzle:
    def __init__(self, row: str):
        fields = row.split(',')
        self.fen = fields[1]
        self.moves = fields[2].split(" ")
        self.tags = fields[7].split(" ")

    def __str__(self):
        return "{fen: " + self.fen + " ,tags: [" + ", ".join(self.tags) + "],moves: [" + ",".join(self.moves) + "]}"

In [7]:
def load(k: int) -> [Puzzle]:
    f = open("lichess_db_puzzle.csv")
    f.readline()
    result = []
    for i in range(k):
        result.append(Puzzle(f.readline()))
    f.close()
    return result

In [8]:
load(10)[0].__str__()

'{fen: r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24 ,tags: [crushing, hangingPiece, long, middlegame],moves: [f2g3,e6e7,b2b1,b3c1,b1c1,h6c1]}'

# FILTER DATASET

In [9]:
expected_tags = {
    'attraction',
    'discoveredAttack',
    'doubleCheck',
    'fork',
    'pin',
    'sacrifice',
    'skewer',
    'xRayAttack',
    'zugzwang',
    'deflection',
    'clearance'
}

In [10]:
expected_tags_list = list(expected_tags)

In [11]:
def filter_data(data: [Puzzle]) -> [Puzzle]:
    return list(filter(lambda p: len(set(p.tags) & expected_tags) == 1, data))

In [12]:
len(filter_data(load(100)))

37

# CONVERSION TO TENSOR

In [13]:
def bitboard_to_tensor(bitboard: int) -> torch.Tensor:
    li = [1 if digit == '1' else 0 for digit in bin(bitboard)[2:]]
    li = [0 for _ in range(64 - len(li))] + li
    return torch.tensor(li).reshape((8, 8))

In [14]:
def fen_to_tensors_list(fen: str) -> [torch.Tensor]:
    board = chess.Board(fen)
    return [
        bitboard_to_tensor(board.occupied_co[chess.WHITE]),
        bitboard_to_tensor(board.occupied_co[chess.BLACK]),
        bitboard_to_tensor(board.pawns),
        bitboard_to_tensor(board.kings),
        bitboard_to_tensor(board.queens),
        bitboard_to_tensor(board.knights),
        bitboard_to_tensor(board.bishops),
        bitboard_to_tensor(board.rooks)
    ]

In [15]:
fen_to_tensors_list(load(1)[0].fen)

[tensor([[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 1, 0, 1, 0],
         [1, 1, 0, 0, 0, 1, 0, 1],
         [1, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[1, 0, 0, 0, 0, 0, 0, 1],
         [1, 0, 0, 1, 0, 0, 1, 1],
         [0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 1, 1],
         [0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [1, 1, 0, 0, 0, 1, 0, 1],
         [0, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0

In [16]:
def move_to_tensor(move: str) -> torch.Tensor:
    x1 = 7 - ord(move[0]) + ord('a')
    y1 = 8 - int(move[1])
    x2 = 7 - ord(move[2]) + ord('a')
    y2 = 8 - int(move[3])
    tensor = torch.zeros(8, 8)
    tensor[y1][x1] = 1
    tensor[y2][x2] = 1
    return tensor

In [18]:
print(move_to_tensor('e2e4'))

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [19]:
def puzzle_to_tensor(puzzle: Puzzle) -> torch.Tensor:
    fen_tensors = fen_to_tensors_list(puzzle.fen)
    move_tensors = [move_to_tensor(puzzle.moves[0]), move_to_tensor(puzzle.moves[1])]  # FIRST TWO MOVES
    return torch.stack(fen_tensors + move_tensors)

In [20]:
puzzle_to_tensor(load(1)[0])

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 1., 0., 1., 0.],
         [1., 1., 0., 0., 0., 1., 0., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 1., 0., 0., 1., 1.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 1., 1.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0., 1., 0., 1.],
       

# CONVERT AND BATCH DATASET

In [21]:
def puzzle_to_truth(puzzle: Puzzle) -> torch.Tensor:
    tensor = torch.zeros(len(expected_tags_list))
    [tag] = set(puzzle.tags) & expected_tags
    index = expected_tags_list.index(tag)
    tensor[index] = 1
    return torch.zeros(1) + index

In [22]:
puzzle_to_truth(filter_data(load(100))[0])

tensor([9.])

In [23]:
BATCH_SIZE = 64

In [24]:
def convert_dataset(puzzles: [Puzzle]) -> list[tuple[torch.Tensor, torch.Tensor]]:
    return [(puzzle_to_tensor(puzzle), puzzle_to_truth(puzzle)) for puzzle in puzzles]

In [25]:
def dataset_to_batches(dataset: list[tuple[torch.Tensor, torch.Tensor]]) -> list[tuple[torch.Tensor, torch.Tensor]]:
    batches = []
    index = 0
    while index + BATCH_SIZE <= len(dataset):
        batch = []
        truth = []
        max_index = index + BATCH_SIZE
        while index < max_index:
            batch.append(dataset[index][0])
            truth.append(dataset[index][1])
            index += 1
        batches.append((torch.stack(batch).cuda(), torch.tensor(truth).cuda().type(torch.long)))

    return batches

In [26]:
batched_dataset=dataset_to_batches(convert_dataset(filter_data(load(10000))))
print(len(batched_dataset))
print(batched_dataset[0][0].shape,batched_dataset[0][1].shape)

47
torch.Size([64, 10, 8, 8]) torch.Size([64])


# TRAIN

In [27]:
def accuracy(out,truth):
    return torch.argmax(out,dim=1) == truth

In [41]:
class Model(nn.Module):
    def __init__(self, *args, **kwargs):
        super(Model, self).__init__()
        self.classifier = nn.Sequential(*args, **kwargs)

    def forward(self, X):
        return self.classifier.forward(X)


model = Model(nn.Conv2d(10, 8 * 8, kernel_size=4, padding=2),
              nn.ReLU(),
              nn.Conv2d(8 * 8, 4 * 4, kernel_size=2, padding=2),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=4, stride=1),
              nn.Conv2d(4*4, 8 * 8, kernel_size=2, padding=2),
              nn.ReLU(),
              nn.Conv2d(8 * 8, 1, kernel_size=4, padding=2),
              nn.ReLU(),
              nn.Flatten(),
              nn.Linear(169, 256),
              nn.ReLU(),
              nn.Linear(256, 64),
              nn.ReLU(),
              nn.Linear(64, 11),
              nn.LogSoftmax(),
              )
criterion = (
    nn.NLLLoss()
)


In [29]:
size_to_load=3000000
test_batches_count=500

In [30]:
all_batches=dataset_to_batches(convert_dataset(filter_data(load(size_to_load))))
train_batches=all_batches[test_batches_count:]
test_batches=all_batches[:test_batches_count]
print(len(all_batches),len(train_batches),len(test_batches))

9421 9021 400


In [42]:
def train(model, criterion, optimizer, epoch):
    model.cuda()
    criterion.cuda()
    batches = train_batches
    size=len(batches)
    print("Dataset size:", len(batches))
    for i in range(epoch):
        time_started = time.time() * 1000
        loss_sum=0.0
        accuracy_sum=0.0
        for batch, truth in batches:
            optimizer.zero_grad()
            out = model.forward(batch)
            loss = criterion(out, truth)
            loss.backward()
            optimizer.step()
            accuracy_value = accuracy(out,truth).sum()/BATCH_SIZE
            
            loss_sum+=loss.item()
            accuracy_sum+=accuracy_value.item()
            
        passed_time = math.ceil(time.time() * 1000 - time_started)
        print(f"Epoch [{i+1}/{epoch}], loss: {loss_sum/size}, accuracy: {accuracy_sum/size}, time: {passed_time/1000}s")

In [43]:
train(model,
      criterion,
      torch.optim.SGD(model.classifier.parameters(), lr=0.001),
      50)

Dataset size: 9021
Epoch [1/50], loss: 1.95151155253443, accuracy: 0.36970333111628423, time: 13.934s
Epoch [2/50], loss: 1.8682542891714342, accuracy: 0.3784953164837601, time: 13.55s
Epoch [3/50], loss: 1.8610261919730757, accuracy: 0.3784953164837601, time: 13.427s
Epoch [4/50], loss: 1.8426527727629443, accuracy: 0.3784953164837601, time: 13.818s
Epoch [5/50], loss: 1.8015298650714302, accuracy: 0.3784953164837601, time: 13.641s
Epoch [6/50], loss: 1.7488994132968965, accuracy: 0.38385953608247425, time: 13.37s
Epoch [7/50], loss: 1.694304491289306, accuracy: 0.4008338183128256, time: 13.421s
Epoch [8/50], loss: 1.6508158516849492, accuracy: 0.4217762720319255, time: 14.199s
Epoch [9/50], loss: 1.6233589556301502, accuracy: 0.4354700144108192, time: 13.324s
Epoch [10/50], loss: 1.6022014827768445, accuracy: 0.44556451612903225, time: 13.372s
Epoch [11/50], loss: 1.584401277200225, accuracy: 0.452735284336548, time: 13.349s
Epoch [12/50], loss: 1.5681747004642492, accuracy: 0.458749

In [ ]:
torch.save(model,'model.pt')

In [44]:
def test(model, criterion):
    model.cuda()
    criterion.cuda()
    batches = test_batches
    print("Dataset size:", len(batches))
    batch_index = 0

    total_loss = 0
    total_accuracy = 0
    for i in range(len(batches)):

        batch = batches[i][0]
        truth = batches[i][1]

        if batch_index == len(batches):
            batch_index = 0

        out = model.forward(batch)
        loss = criterion(out, truth)
        print(f"Batch [{i+1}/{len(batches)}], loss: {loss.item()}, accuracy: {(accuracy(out,truth).sum()/BATCH_SIZE).item()}")

        total_loss += loss.item()
        total_accuracy+=(accuracy(out,truth).sum()/BATCH_SIZE).item()

    return (total_loss / len(batches)),total_accuracy / len(batches)

In [45]:
test(model, criterion)

Dataset size: 400
Batch [1/400], loss: 1.2641535997390747, accuracy: 0.5625
Batch [2/400], loss: 1.2908371686935425, accuracy: 0.5625
Batch [3/400], loss: 1.1458052396774292, accuracy: 0.640625
Batch [4/400], loss: 1.13771653175354, accuracy: 0.625
Batch [5/400], loss: 1.3008922338485718, accuracy: 0.53125
Batch [6/400], loss: 1.3089154958724976, accuracy: 0.5
Batch [7/400], loss: 1.2504442930221558, accuracy: 0.5625
Batch [8/400], loss: 1.235909342765808, accuracy: 0.59375
Batch [9/400], loss: 1.291830062866211, accuracy: 0.578125
Batch [10/400], loss: 1.1903491020202637, accuracy: 0.640625
Batch [11/400], loss: 1.7065924406051636, accuracy: 0.4375
Batch [12/400], loss: 1.2043167352676392, accuracy: 0.59375
Batch [13/400], loss: 1.4448678493499756, accuracy: 0.484375
Batch [14/400], loss: 1.194724678993225, accuracy: 0.5
Batch [15/400], loss: 1.4067864418029785, accuracy: 0.5
Batch [16/400], loss: 1.3691610097885132, accuracy: 0.46875
Batch [17/400], loss: 1.3388104438781738, accuracy

(1.300299287736416, 0.5600390625)

In [ ]:
def eye_test(model,puzzle):
    tensor=puzzle_to_tensor(puzzle).cuda()
    out=model.forward(tensor)
    return sorted(zip(expected_tags_list,out.squeeze().tolist()),key=lambda x:-x[1]),puzzle.tags

eye_test(model,filter_data(load(100))[0])